In [1]:
import os

In [2]:
os.chdir('..')

In [3]:
%pwd

'c:\\Users\\Kushal\\Documents\\GitHub\\kidney-disease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    base_model_path: Path
    trained_model_path: Path
    training_data_path: Path
    params_num_epochs: int
    params_learning_rate: float
    params_batch_size: int

In [6]:
from classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from classifier.utils.common import read_yaml, create_directories

In [7]:
import torch

In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training
        base_model_config = self.config.prepare_base_model
        data_ingestion_config = self.config.data_ingestion
        params = self.params
        create_directories([training_config.root_dir])

        training_data_path_ = os.path.join(self.config.data_ingestion, 'kidney-ct-scan-image')

        training_config = TrainingConfig(
            root_dir = Path(training_config.root_dir),
            base_model_path = Path(base_model_config.base_model_path),
            trained_model_path = Path(training_config.trained_model_path),
            training_data_path = Path(training_data_path_),
            params_num_epochs = params.EPOCHS,
            params_learning_rate = params.LEARNING_RATE,
            params_batch_size = params.BATCH_SIZE
            )
        
        return training_config


    # base_model_path: Path
    # trained_model_path: Path
    # params_num_epochs: int
    # params_learning_rate: float
